In [ ]:
# ARCHIVED -- did not find a trend

In [65]:
from utils import *
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter



# from 'get_inaccurate_and_all_alleles' (utils.py) modified to return pairs instead of zerodig & allalleles
# returns only the first allele in the gold standard if there are multiple 
# returns the closest prediction, if the gold standard is monoallelic
# also add a few lines to remove pairs containing 'nan' from the return
def get_mispredicted_pairs(pre,gs):

    pairs = []
    fail = 0

    accession_numbers = gs['Run'].values.tolist()
    genes = gs.columns.values.tolist()

    for number in accession_numbers:
        pre_row = pre.loc[pre['ERR'] == number]
        gs_row = gs.loc[gs['Run'] == number]
        
        
        # if we are working with d5 or d6, the monoallelic datasets
        if (gs.columns.tolist() == ['Run', 'A', 'B'] or gs.columns.tolist() == ['Run', 'C']):
            for i in range(1,len(genes)):
                gs_val = gs_row[genes[i]].astype(str).values[0]
                pre_val1 = pre_row[genes[i]].astype(str).values[0]
                pre_val2 = pre_row[genes[i]+".1"].astype(str).values[0]
                
                # if the gold standard contains many allele possibilities, and the caller is incorrect,
                # we will return only the first value in the gs
                gs_primary = reformat( gs_val.split("/")[0])

                ans1 = compute_resolution(gs_val,pre_val1)
                ans2 = compute_resolution(gs_val,pre_val2)
                if (gs_val != 'nan' and max(ans1,ans2) == 0):
                    pre_primary = min([pre_val1,pre_val2], key = lambda w: levenshtein(fix(gs_primary), w))
                    if (pre_primary != 'nan'):
                        pairs.append(gs_primary + " " + pre_primary)


        # if we are working with d1-d4, the biallelic datasets
        else:
            for i in range(1,len(genes),2):
                try:
                    gs_val1 = gs_row[genes[i]].astype(str).values[0]
                    pre_val1 = pre_row[genes[i]].astype(str).values[0]
                    gs_val2 = gs_row[genes[i+1]].astype(str).values[0]
                    pre_val2 = pre_row[genes[i+1]].astype(str).values[0]

                    if (gs_val1 == None) or (pre_val1 == None) or (gs_val2 == None) or (pre_val2 == None):
                        fail = fail+1
                        continue
                        
                    # if the gold standard contains many allele possibilities, and the caller is incorrect,
                    # we will return only the first value in the gs
                    gs_primary1 = reformat( gs_val1.split("/")[0] ) 
                    gs_primary2 = reformat( gs_val2.split("/")[0] ) 

                    # assuming no swapping 
                    ans1 = compute_resolution(gs_val1,pre_val1)
                    ans2 = compute_resolution(gs_val2,pre_val2)

                    # assuming swapping
                    ans3 = compute_resolution(gs_val1,pre_val2)
                    ans4 = compute_resolution(gs_val2,pre_val1)
                    
                    if pre_val1 != 'nan' and pre_val2 != 'nan':

                        if (ans1+ans2 > ans3+ans4):
                            if (ans1 == 0):
                                pairs.append(gs_primary1 + " " + pre_val1)
                            if (ans2 == 0):
                                pairs.append(gs_primary2 + " " + pre_val2)
                        else:
                            if (ans3 == 0):
                                pairs.append(gs_primary1 + " " + pre_val2)
                            if (ans4 == 0):
                                pairs.append(gs_primary2 + " " + pre_val1)
                except:
                    fail = fail+1

    return pairs # pairs of [gs, pre]


In [66]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]


In [83]:
data = list() 
tools=["optitype","arcas","rna2hla","hlaforest","seq2hla","hlahd","phlat"] #removed hlaminer, hlavbseq

for t in tools:
    total = []
    for d in range(1,7):
        gs=pd.read_csv("../datasets/"+str(d)+"_gs.csv")
        pre=pd.read_csv("../results/standard/"+str(t)+"_d"+str(d)+".csv")
        ret=get_mispredicted_pairs(pre,gs)
        total.extend(ret)
    data.extend(total)
    


In [84]:
print(data)


['B*1: B*57:01', 'B*15:01 B*51:01', 'B*40:01 B*35:03', 'C*03:04 C*04:01', 'A*68:01 A*31:01', 'B*44:02 B*07:02', 'B*40:01 B*15:01', 'B*15:01 B*51:01', 'B*40:01 B*35:03', 'C*03:04 C*04:01', 'A*02:01 A*03:01', 'B*07:02 B*39:06', 'C*07:02 C*15:02', 'B*15:01 B*56:03', 'A*01:01 A*03:01', 'B*08:01 B*07:02', 'B*55:01 B*39:06', 'C*07:01 C*15:02', 'B*1: B*57:01', 'C*07:02 C*04:01', 'B*07:02 B*39:06', 'C*07:02 C*04:01', 'C*07:02 C*04:01', 'B*15:01 B*51:01', 'A*24:24 A*23:13', 'A*03:01 A*24:18', 'C*12:03 C*02:205Q', 'A*02:01 A*23:01', 'B*07:02 B*39:24', 'A*02:01 A*23:01', 'B*07:02 B*13:02', 'B*07:02 B*56:01', 'A*24:02 A*01:01', 'DQB1*06:03 DQB1*05:03', 'A*02:01 A*69:01', 'C*07:02 C*15:13', 'B*35:01 B*67:01', 'C*12:03 C*04:431', 'A*02:01 A*23:01', 'C*07:02 C*15:05', 'A*01:01 A*02:05', 'C*07:01 C*04:01', 'C*07:02 C*04:01', 'A*24:02 A*02:844', 'C*12:03 C*04:431', 'B*40:01 B*35:03', 'C*03:04 C*04:01', 'A*02:01 A*68:01', 'C*05:01 C*04:01', 'DRB1*08:04 DRB1*13:03', 'A*68:01 A*31:01', 'B*44:02 B*15:01', 

In [86]:
counts = Counter(data)

df = []
for k in counts.keys():
    l = k.split(' ')
    df.append([l[0],l[1],counts[k]])

df = pd.DataFrame(df,columns=['gold standard allele','prediction','count'])
df = df.sort_values(by=['count'],ascending=False)
print(df.head(30))

    gold standard allele     prediction  count
53            DRB1*07:01    DRB1*14:103     43
39            DQB1*02:01     DQB1*06:02     29
15               C*07:02        C*04:01     27
57            DRB1*15:01     DRB1*13:07     24
287           DQB1*02:01  DQB1*03:01:01     23
301           DQB1*02:01  DQB1*05:01:01     22
267           DQB1*02:01  DQB1*06:02:01     20
296           DQB1*02:01  DQB1*03:02:01     17
114           DQB1*04:02     DQB1*03:05     14
315           DQB1*03:01  DQB1*06:02:01     14
41            DRB1*01:01     DRB1*04:01     14
295           DQB1*03:01  DQB1*05:01:01     14
60               B*53:01        B*35:01     13
338           DQB1*02:01  DQB1*04:02:01     13
297           DQB1*03:02  DQB1*05:01:01     13
3                C*03:04        C*04:01     12
303           DQB1*02:01  DQB1*03:03:02     11
1                B*15:01        B*51:01     11
299           DQB1*03:01     DQB1*06:02     10
291           DQB1*02:01  DQB1*06:03:01     10
2            

future steps:
for each of the top mispredicted pairs, determine which sample(s) the misprediction came from -- extract the reads from those samples -- use alignment software to visualize sequence similarity